### Query ESO archive and calculate total time for each PID
#### in any given time frame

In [12]:
import astroquery
from astroquery.eso import Eso
import pandas as pd
import numpy as np
import sys
from numpy import *
from astropy.table import Table
from astropy.table import *
from astropy.io import ascii


eso=Eso()
eso.ROW_LIMIT = -1 


pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns',None)

### Defines a general search for available data in the ESO archive
#### - using  a date range, PIDs and instrument as input

In [13]:
def eso_query(instr,start,end,pid):
#    print("Check PID: ",pid)
    num_obs= 0
    tot_exptime = 0.0
    table = eso.query_main(column_filters={'instrument':instr,'dp_cat': 'SCIENCE','stime':start,'etime':end,'prog_id':pid})
    if not table:
        num_Obs      = num_obs + 0
        tot_exptime  = 0.0
    else:
        num_Obs = num_obs + len(table)
        
        pd_table    = table.to_pandas()
        tot_exptime = pd_table["Exptime"].sum()
    
    return num_Obs,tot_exptime


def eso_query_grond(instr,start,end,pid):

    num_ir = 0
    num_opt=0
    num_obs = 0
#    tot_exptime=0
    table = eso.query_main(column_filters={'instrument': instr, 'prog_id':pid,'dp_cat': 'SCIENCE','stime':start,'etime':end} )
        
    if not table: 

#        print("No data found for:",pid )
            
        num_opt = 0
        num_nir  = 0
        tot_exptime = 0.0
        tot_time = 0.0
        
    else:
# Check for optical images        
        lambda_lo = 1100
        table_opt = table[table['filter_lambda_min'] < lambda_lo] # limit list to only optical files
        pd_table  = table_opt.to_pandas()
        num_opt   = len(table_opt)                                     # number of optical images

#        table = table[table_opt['Exptime'] > 5.0] # limit list to NIR exptimes > 2 sec
#        print("Number of optical files :",len(table_opt))
        
        table_nir = table[table['filter_lambda_min'] > lambda_lo]      # number of near-infrared images
        pd_table    = table_nir.to_pandas()
        tot_exptime = (6*pd_table["Exptime"].sum()/3600.0)
#        print("Number of NIR files: ", len(table_nir))
        num_nir = len(table_nir)


        num_obs = num_opt + num_nir
    return num_obs,tot_exptime
    

In [54]:
from datetime import datetime, date, timedelta
info = "/home/angela/LaSilla/P114/P114_pi.list"

data = open(info,'r')

#start   = input('Start date of query (yyyy-mm-dd):  ')
#end     = input('End date of query (yyyy-mm-dd):    ')
start = "2024-10-01"

end = (datetime.now()+timedelta(days=2)).strftime('%Y-%m-%d')             # date for tomorrow, to make sure that all nights are  used
#end = "2024-12-02"
print(end)

end_f   = (datetime.now() + timedelta(days=-1) ).strftime('%Y-%m-%d')     # date for yesterday, the day the last observing night started
#end_f   = end.strftime('%Y-%m-%d')

print("     ")
print("     ")
print("     ")

orig_stdout = sys.stdout

path = "/home/angela/LaSilla/P114/"

file_out   = path + "P114_stats.cvs"
file_out_2 = path + "P114_stats_a.cvs"

data_2 = []
col1=[]    # full name
col2=[]    # list of PIDs
col3=[]    # instrument
col4=[]    # allocated time in hours Category A
col5=[]    #  requested time in proposal
col6=[]    # number of files
col7=[]    # exposure times
col8=[]    # execution time time in hr
col9=[]    # fraction in %
col10 =[]  # fraction completed in of total %


data_3 = []
cola=[]    # full name
colb=[]    # 1st PID
colc=[]    # 2nd PID
cold=[]    # instrument
cole=[]    # allocated time A in hr
colf=[]    # tie asked for in proposal
colg=[]    # number of files
colh=[]    # exposure time in hr
coli=[]    # observed time in hr
colj=[]    # fraction completed in %
colk=[]    # fraction of total time asked for in proposal

tot_exptime = 0.0
header1 = data.readline()
header2 = data.readline()
for line in data:
    
    num = 0
    tot_exptime = 0.0
    tot_time    = 0.0
    line     = line.strip()
    columns  = line.split()
    name     = columns[0]
    first    = columns[1]
    email    = columns[2]
    pid_1    = columns[3]
    pid_2    = columns[4]
    instr    = columns[5]
    t_tot_A  = float(columns[6])    
    t_tot_B  = float(columns[7])
    
    f_name = name+','+ first
    full_name = ("{:24}".format(f_name))

    idlist = pid_1
    if pid_2 != str("000.0000.000"):
        idlist = idlist+", "+pid_2
        


# define overheads per image    
    if instr  == "FEROS":
#        num = 0
#        tot_exptime = 0.0
        result_1 = eso_query(instr,start,end,pid_1)
        num = num+result_1[0]
        tot_exptime= tot_exptime+result_1[1]

        result_2 = eso_query(instr,start,end,pid_2)
        num= num + result_2[0]
#        print(num)
        tot_exptime = tot_exptime+result_2[1]
        oh_time = 282.0
        tot_exptime = tot_exptime/3600.0
        tot_time    = ((num*oh_time)/3600.0)  + tot_exptime


        if name== "COSTA":                    # usual does 2 exposures in 1 OB, overheads are 05:24 min=324 sec
            num=0
            tot_exptime= 0.0
            result_1 = eso_query(instr,start,end,pid_1)
#            print(result_1[0])
            num = num+result_1[0]
            tot_exptime= (tot_exptime+result_1[1])
            result_2 = eso_query(instr,start,end,pid_2)
            num = num+result_2[0]
            tot_exptime= (tot_exptime+result_2[1])/3600.0

            tot_time = ((num/2)*(324.0)/3600.0)+(tot_exptime)
#        print("Finished  ",name)
    if instr == "WFI" :
        if name =="BANADOS":
            tot_exptime = 0
            num = 0
            overhead = 16.10*60.0    # 16:10 min     
            result_1 = eso_query(instr,start,end,pid_1)
            num = num+result_1[0]
            tot_exptime= (tot_exptime+result_1[1])
            result_2 = eso_query(instr,start,end,pid_2)
            num = num + result_2[0]
            
            tot_exptime= (tot_exptime+result_2[1])/3600.0
            
            tot_time    = ((num/4)*overhead)/3600.0 + tot_exptime


 
        if name =="NEUMANN":
            num=0
            tot_exptime = 0
            overhead    = 700.0
            result_1    = eso_query(instr,start,end,pid_1)
            num         = num+result_1[0]
            tot_exptime = (tot_exptime + result_1[1])/3600.00
            tot_time    = tot_exptime + (num*70.0)/3600.0
            print(num)
        if name=="STANKE":
            num=0
            tot_exptime = 0
            start_n       ="2024-11-23"
            result_1    = eso_query(instr,start_n,end,pid_1)
            num         = num+result_1[0]
            overheads   =   (num/4)*(19*60.0)       # 19 min overheads per OB (1 filter), which includes 4 exposures
            tot_exptime = (tot_exptime+result_1[1])/3600.0
            tot_time    = tot_exptime+(overheads)/3600.0
            


    
    if instr == "GROND":
        num           = 0
        overhead      = 0.0
        tot_exptime   = 0.0                                              # exposure time
        tot_time      = 0.0                                              

        result_1      =   eso_query_grond(instr,start,end,pid_1)
        num           = num + result_1[0]
        tot_exptime   = tot_exptime + result_1[1]

# In case a second PID was used, check that one as well     
        
        result_2 = eso_query_grond(instr,start,end,pid_2)
        num = num + result_2[0]
        tot_exptime = tot_exptime + result_2[1]
        tot_time = (tot_exptime+ num*overhead)                           # for GROND overheads are basically = 0

        
        
    if tot_exptime > 0.0:
        time_perc_A   =  100.0*(tot_time/(t_tot_A+0.001))                  # percent of allocated Cat A time observed
        time_perc_B   =  100.0*(tot_time/(t_tot_B+0.001))                  # percent of total time asked for
    if tot_exptime  == 0.0:
        time_perc_A   = 0.0
        time_perc_B   = 0.0
        
    t_tot_A_f    = ("{:>4.1f}".format(t_tot_A))                                # formatted allocated time cat- A
    t_tot_B_f    = ("{:>4.1f}".format(t_tot_B))                                # formatted allocated time cat- B
    tot_time_f = ("{:>4.1f}".format(tot_time))                                 # formatted observed time
    time_perc_A_f=  ("{:>6.1f}".format(time_perc_A))
    time_perc_B_f=  ("{:>6.1f}".format(time_perc_B))
    tot_exptime_f = ("{:>4.1f}".format(tot_exptime))
    

    data_2.append({'#PI_name                ': full_name.ljust(20," "),
                   'PIDs              ': idlist.ljust(30," "),
                   'Inst.': str(instr).center(5," "),
                   'Cat-A [h]': str(t_tot_A_f).rjust(10," "),
                   'total [h]': str(t_tot_B_f).rjust(10," "),
                   '# files': str(num).rjust(10," "),
                   'exp. [h]': '%6.1f' %tot_exptime,
                   'exec. [h] ': str(tot_time_f).center(10," "),
                   '  A [%]': time_perc_A_f,
                   ' tot. [%]': time_perc_B_f
        }              
    )
    col1.append(full_name)
    col2.append(idlist.ljust(30," "))
    col3.append(instr.center(9," "))
    col4.append(t_tot_A_f)
    col5.append(t_tot_B_f)
    col6.append(num)
    col7.append(tot_exptime_f)
    col8.append(tot_time_f)
    col9.append(time_perc_A_f)
    col10.append(time_perc_A_f)
              
    data_3.append({'#PI_name': full_name.ljust(20," "),
                   '       PID_1       ': pid_1.center(15," "),
                   '       PID_2       ': pid_2.center(15," "),
                   'Instrument  ': str(instr).center(5," "),
                   'Cat-A [h]': str(t_tot_A_f).rjust(10," "),
                   'total [h]': str(t_tot_B_f).rjust(10," "),
                   '# of files': str(num).rjust(10," "),
                   'exp. [h]': '%6.1f' %tot_exptime,
                   'exec.[h] ': str(tot_time_f).center(10," "),
                   'compl. A [%]': time_perc_A_f,
                   'compl. tot.[%]': time_perc_B_f
        }              
    )
    cola.append(full_name)
    colb.append(pid_1.center(15," "))
    colc.append(pid_2.center(15," "))
    cold.append(instr.center(15," "))
    cole.append(t_tot_A_f)
    colf.append(t_tot_B_f)
    colg.append(num)
    colh.append(tot_exptime_f)
    coli.append(tot_time_f)
    colj.append(time_perc_A_f)
    colk.append(time_perc_B_f)


pd.set_option('display.precision', 1)    
df = pd.DataFrame(data_2)
df_1 = pd.DataFrame(data_3)
number = len(df.index)

print(df.to_string(index=False))
#print(new)
li = [df.columns.values.tolist()] + df.values.tolist()
df.to_csv(file_out, quoting=None,index=False) 
df_1.to_csv(file_out_2, index=False) 
print("    ")
print("    ")

print("Finished")
sys.stdout=orig_stdout


2024-12-04
     
     
     


68
#PI_name                             PIDs               Inst.  Cat-A [h]  total [h]    # files exp. [h] exec. [h]    A [%]  tot. [%]
RAU,Arne                 0113.A-3003(A), 114.27S7.001   GROND      162.0      162.0        351      5.2     5.2        3.2       3.2
DEMIANENKO,Mariia        114.27SY.001                   GROND       38.0       38.0        110      1.6     1.6        4.2       4.2
WANG,Lingzhi             114.27SQ.001                   GROND       15.0       15.0          0      0.0     0.0        0.0       0.0
SUYU,Sherry              114.27ST.003                   GROND       10.0       10.0          0      0.0     0.0        0.0       0.0
RAU,Arne                 114.27S9.001, 114.24SB.001     FEROS        0.1        0.1          0      0.0     0.0        0.0       0.0
MANCINI,Luigi            114.27SG.001                   FEROS        2.0        2.0          2      0.5     0.7       32.8      32.8
SEEBURGER,Rhys           114.27T7.001                   FEROS     

### Detailed search, also including corrections for:
#### - repeated observations, e.g. Neuman
#### - excluding test runs, e.g. for GROND
#### - only counting NIR images for GROND
#### - specific overheads for WFI programs with time-consuming filter changes, e.g. Bañados

### Write a pdf file with results

In [55]:
import reportlab
from reportlab.pdfgen import canvas
from reportlab.lib.pagesizes import letter,A4
from reportlab.lib.units import inch
from reportlab.lib import colors
import os
from reportlab.platypus import SimpleDocTemplate, Table, TableStyle,Paragraph, Spacer
from reportlab.lib.styles import ParagraphStyle,getSampleStyleSheet
from reportlab.lib.units import cm

styles = getSampleStyleSheet()

path = "/home/angela/LaSilla/P114/"
date_now = (datetime.now()).strftime('%Y-%m-%d')
pdf_out = path + end_f+".pdf"
print(pdf_out)
doc = SimpleDocTemplate(pdf_out, pagesize=A4,fontsize=8)
# container for the 'Flowable' objects
elements = []



colw = (1*inch,1.6*inch,0.55*inch,0.7*inch, 0.7*inch, 0.7*inch, 0.7*inch,0.7*inch, 0.7*inch,0.7*inch)
t=Table(li,colw,(number+1)*[0.3*inch])

t.setStyle(TableStyle([('ALIGN',(1,1),(8,number),'CENTER'),
                       ('ALIGN',(1,1),(1,number),'LEFT'),
                       ('FONTSIZE',(0,0),(10,number),5),
                       ('TEXTCOLOR',(1,1),(10,number),colors.black),
                       ('TEXTCOLOR',(0,0),(0,-1),colors.blue),
                       ('TEXTCOLOR',(0,16),(10,16),colors.red),
                       ('BACKGROUND', (0, 0), (0, number), colors.lightblue),
                       ('BACKGROUND', (1, 0), (10, 0), colors.lightblue),
                       ('BACKGROUND',(0,number-3),(0,number),colors.orange),
                       ('VALIGN',(0,-1),(-1,-1),'MIDDLE'),
                       ('INNERGRID', (0,0), (-1,-1), 0.25, colors.black),
                       ('BOX', (0,0), (-1,-1), 0.25, colors.black),
                       ]))
elements.append(t)


title = "P114 from " + start + " until "+end_f

flowables = [
    Paragraph(title, styles['Title']),
    t,
    Spacer(1 * cm, 1 * cm),
#    Paragraph('Programs for Moyano,Vines and Anderson use different PIDs for each run. The total allocated time for Chile programs: 14 nights (I assume 10 hr per night)')
Paragraph('* Semenov does not have a program ID yet!!!')
]
doc.build(flowables)





/home/angela/LaSilla/P114/2024-12-01.pdf


### Statistic for any given night and PID

In [16]:
pid= "113.26Y5"   # Villasenor

instr ="FEROS"
start = "2024-09-17"
end   = "2024-09-19"

oh = 282
#oh = 120.0    # 3 exposures in on OB, averaged
num= 0
exp_time=0.0
result    = eso_query(instr,start,pid)

num       = result[0]
exp_time  = result[1]
exec_time = ((oh*num)+exp_time)/3600.0

seconds = int(exec_time*3600.0)
seconds = seconds % (24 * 3600)
hour = seconds // 3600
seconds %= 3600
minutes = seconds // 60
seconds %= 60

print(end)
print("Number of files: ",num)
print('{:10}{:02d}{:1}{:02d}'.format(pid,hour, ":",minutes))


2024-09-19
Number of files:  2
113.26Y5  00:36


In [11]:
print(end_f)

2024-12-01


In [37]:
print(li)

[['#PI_name                ', 'PIDs              ', 'Inst.', 'Cat-A [h]', 'total [h]', '# files', 'exp. [h]', 'exec. [h] ', '  A [%]', ' tot. [%]'], ['RAU,Arne                ', '0113.A-3003(A), 114.27S7.001  ', 'GROND', '     162.0', '     162.0', '       351', '   5.2', '    5.2   ', '   3.2', '   3.2'], ['DEMIANENKO,Mariia       ', '114.27SY.001                  ', 'GROND', '      38.0', '      38.0', '       110', '   1.6', '    1.6   ', '   4.2', '   4.2'], ['WANG,Lingzhi            ', '114.27SQ.001                  ', 'GROND', '      15.0', '      15.0', '         0', '   0.0', '    0.0   ', '   0.0', '   0.0'], ['SUYU,Sherry             ', '114.27ST.003                  ', 'GROND', '      10.0', '      10.0', '         0', '   0.0', '    0.0   ', '   0.0', '   0.0'], ['RAU,Arne                ', '114.27S9.001, 114.24SB.001    ', 'FEROS', '       0.1', '       0.1', '         0', '   0.0', '    0.0   ', '   0.0', '   0.0'], ['MANCINI,Luigi           ', '114.27SG.001              